In [28]:
# -*- coding: utf-8 -*-
"""Untitled1.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/12eMKUl03r3KesaaCxjqNaU8JNEVzFxCZ
"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix # Import classification_report and confusion_matrix

#trattamento dei dati
#trasformare le 4 colonne con stringhe in numerico
def cleanData(data) :
    for col in data.columns.values:
        data[col] = data[col].astype('string')
    #----------
    for col in data.columns.values:
        data[col] = data[col].astype('float',errors = 'ignore')
    #-----------
    data['Gender']=data.Gender.map({'Male': 1, 'Female': 2})
    data['Dyslexia']=data.Dyslexia.map({'No': 0, 'Yes': 1})
    data['Nativelang']=data.Nativelang.map({'No': 0, 'Yes': 1})
    data['Otherlang']=data.Otherlang.map({'No': 0, 'Yes': 1})

desktopData= pd.read_csv("/content/Dyslexia_dataset.csv", sep = ';', encoding = 'utf-8')
cleanData(desktopData)

commonalityColumns = ['Gender','Nativelang','Otherlang','Age' , 'Dyslexia']
for i in  range(30):
    if((i>=0 and i<12) or (i>=13 and i<17) or i==21 or i==22 or i==29):
        commonalityColumns.append('Clicks'+str(i+1))
        commonalityColumns.append('Hits'+str(i+1))
        commonalityColumns.append('Misses'+str(i+1))
        commonalityColumns.append('Score'+str(i+1))
        commonalityColumns.append('Accuracy'+str(i+1))
        commonalityColumns.append('Missrate'+str(i+1))

desktopData=desktopData.loc[:,commonalityColumns]

y=desktopData['Dyslexia']
X=desktopData.loc[:, desktopData.columns != 'Dyslexia']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier


sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)


In [33]:
from sklearn.model_selection import GridSearchCV

# Definizione della griglia di parametri da ottimizzare per Grid Search
param_grid = {
    'C': [0.1, 1, 10, 100 ],               # Parametro di regolarizzazione
    'gamma': [1e-4, 1e-3, 0.01, 0.1, 1 , 10 , 100],       # Parametro del kernel
    'kernel': ['rbf', 'linear']           # Tipi di kernel
}


In [32]:
from sklearn.svm import SVC

svm_clf = SVC(
    kernel='rbf',
    class_weight='balanced',
    probability=True,
    random_state=42
)
svm_clf.fit(X_train_bal, y_train_bal)
y_pred_svm = svm_clf.predict(X_test)

print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm))

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.47      0.63       649
           1       0.17      0.85      0.28        80

    accuracy                           0.51       729
   macro avg       0.56      0.66      0.46       729
weighted avg       0.87      0.51      0.60       729

Confusion Matrix:
[[307 342]
 [ 12  68]]
